In [2]:
import os

os.listdir('../dataset')

['yes', 'no']

In [4]:
training_dir = '../dataset'

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(300, 300),
    class_mode='binary'
)

Found 253 images belonging to 2 classes.


In [9]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(train_generator, epochs=15)

Epoch 1/15
8/8 [==============================] - 11s 1s/step - loss: 0.7200 - accuracy: 0.6561
Epoch 2/15
8/8 [==============================] - 9s 1s/step - loss: 0.6185 - accuracy: 0.6522
Epoch 3/15
8/8 [==============================] - 9s 1s/step - loss: 0.5460 - accuracy: 0.7352
Epoch 4/15
8/8 [==============================] - 9s 1s/step - loss: 0.5039 - accuracy: 0.7787
Epoch 5/15
8/8 [==============================] - 9s 1s/step - loss: 0.5006 - accuracy: 0.7668
Epoch 6/15
8/8 [==============================] - 9s 1s/step - loss: 0.5010 - accuracy: 0.7826
Epoch 7/15
8/8 [==============================] - 9s 1s/step - loss: 0.4454 - accuracy: 0.8024
Epoch 8/15
8/8 [==============================] - 9s 1s/step - loss: 0.4145 - accuracy: 0.7984
Epoch 9/15
8/8 [==============================] - 9s 1s/step - loss: 0.4203 - accuracy: 0.8261
Epoch 10/15
8/8 [==============================] - 10s 1s/step - loss: 0.3507 - accuracy: 0.8340
Epoch 11/15
8/8 [==============================